In [ ]:
# Read environment variables

import os
import traceback

import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

openai_url      = os.environ['AZURE_OPENAI_URL']
openai_key      = os.environ['AZURE_OPENAI_KEY']
openai_version  = os.environ['AZURE_OPENAI_API_VERSION']  # "2023-07-01-preview" 
embeddings_dep  = os.environ['AZURE_OPENAI_EMBEDDINGS_DEP']
completions_dep = os.environ['AZURE_OPENAI_COMPLETIONS_DEP']

openai.api_type = "azure"
openai.api_version = openai_version
openai.api_base = openai_url
openai.api_key  = openai_key

print("openai_url:      {}".format(openai_url))
print("openai_key:      {}".format(openai_key[0:3]))
print("openai_version:  {}".format(openai_version))
print("embeddings_dep:  {}".format(embeddings_dep))
print("completions_dep: {}".format(completions_dep))

try:
    aoai_client = AzureOpenAI(
        azure_endpoint=openai_url,
        api_key=openai_key,
        api_version=openai_version,
    )
    print("aoai_client created")
except Exception as e:
    print("Exception while creating AzureOpenAI client: {}".format(str(e)))
    print(traceback.format_exc())


In [ ]:
# Generate embeddings for a given string value.

import traceback
from pysrc.util.fs import FS

try:
    text = "A long time ago came a man on a track. Walking thirty miles with a sack on his back."
    response = aoai_client.embeddings.create(input=text, model=embeddings_dep)
    embedding = response.data[0].embedding

    print("embedding length: {}".format(len(embedding)))   
    print("embedding[:4]: {}".format(embedding[:4])) 
    FS.write_json(embedding, "tmp/embedding.json")
except Exception as e:
    print("Exception while creating embedding: {}".format(str(e)))
    print(traceback.format_exc())


In [ ]:
# Generate a simple completion 

import traceback

try:
    response = aoai_client.chat.completions.create(
        model=completions_dep,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Knock knock."},
            {"role": "assistant", "content": "Who's there?"},
            {"role": "user", "content": "Orange."}
        ],
        temperature=0
    )
    print(response)
    print("role: {} -> {}".format(response.choices[0].message.role, response.choices[0].message.content))
    
    response = aoai_client.chat.completions.create(
        model=completions_dep,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the capitol of North Carolina?"},
        ],
        temperature=0
    )
    print(response)
    print("role: {} -> {}".format(response.choices[0].message.role, response.choices[0].message.content))
    
except Exception as e:
    print("Exception while creating completion: {}".format(str(e)))
    print(traceback.format_exc())

In [8]:
# Generate a python module 

import traceback

try:
    prompt = """
You have two tasks to execute:
- Generate a python 3 class named FSUtil in module pysrc.util.fsutil.py.
- Also generate a unit test for this class which uses the pytest testing framework.

The focus of class FSUtil is on providing easy to use filesystem access to read and write files.

The class has no instance methods, only class methods.

All methods should assume the following:
- Use "utf-8" encoding

It should implement the following list of methods per these signatures:
- def as_unix_filename(cls, filename: str) -> str:
- def read(cls, infile: str) -> str | None:
- def read_with_encoding, infile: str, encoding: str) -> str | None:
- def read_lines(cls, infile: str) -> list[str] | None:
"""
    response = aoai_client.chat.completions.create(
        model=completions_dep,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who generates Python code."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    print(response)
    print("role: {}\n{}".format(response.choices[0].message.role, response.choices[0].message.content))

except Exception as e:
    print("Exception while creating completion: {}".format(str(e)))
    print(traceback.format_exc())

ChatCompletion(id='chatcmpl-9TxynP8KjYDl1qgqxhzyevFyqjlz3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To accomplish your tasks, I\'ll first provide the Python code for the `FSUtil` class within the specified module structure. Then, I\'ll create a basic unit test using the pytest framework.\n\n### Step 1: Creating the FSUtil Class\n\nFirst, ensure you have the correct directory structure. You should create a directory named `pysrc`, within it another directory named `util`, and inside this, a file named `fsutil.py`. The path will be `pysrc/util/fsutil.py`.\n\n**pysrc/util/fsutil.py:**\n\n```python\nimport os\n\nclass FSUtil:\n    @classmethod\n    def as_unix_filename(cls, filename: str) -> str:\n        return filename.replace("\\\\", "/")\n\n    @classmethod\n    def read(cls, infile: str) -> str | None:\n        try:\n            with open(infile, \'r\', encoding=\'utf-8\') as file:\n                return file.read()\n      